In [1]:
from os import path
from fastai.structured import *
from fastai.column_data import *
import pandas as pd # to manipulate data frames 
import numpy as np # to work with matrix

D:\python\envs\fastai\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
PATH=Path('data/kg-google/')

In [3]:
train_file_name = f'{PATH}\extracted_fields_train.csv'
test_file_name= f'{PATH}\extracted_fields_test.csv'

In [5]:
chunksize=24000
columns_to_read = ['channelGrouping', 'date', 'fullVisitorId', 'totals.hits',
       'visitStartTime', 'device.browser', 'device.deviceCategory',
       'device.isMobile', 'device.operatingSystem', 'geoNetwork.city',
       'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.region',
       'geoNetwork.subContinent', 'totals.bounces', 'totals.hits',
       'totals.newVisits', 'totals.pageviews', 'totals.transactionRevenue']

In [6]:
df_train = pd.read_csv(train_file_name, dtype={'fullVisitorId': 'str'},
                      usecols=columns_to_read,low_memory=False
                      )

In [16]:
df_train.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'visitStartTime',
       'device.browser', 'device.deviceCategory', 'device.isMobile',
       'device.operatingSystem', 'geoNetwork.city', 'geoNetwork.continent',
       'geoNetwork.country', 'geoNetwork.region', 'geoNetwork.subContinent',
       'totals.bounces', 'totals.hits', 'totals.newVisits', 'totals.pageviews',
       'totals.transactionRevenue'],
      dtype='object')

In [17]:
# df_train = pd.read_csv(train_file_name)
df_test = pd.read_csv(test_file_name, dtype={'fullVisitorId': 'str'},  usecols=columns_to_read,low_memory=False)

In [18]:
 df_test.columns


Index(['channelGrouping', 'date', 'fullVisitorId', 'visitStartTime',
       'device.browser', 'device.deviceCategory', 'device.isMobile',
       'device.operatingSystem', 'geoNetwork.city', 'geoNetwork.continent',
       'geoNetwork.country', 'geoNetwork.region', 'geoNetwork.subContinent',
       'totals.bounces', 'totals.hits', 'totals.newVisits', 'totals.pageviews',
       'totals.transactionRevenue'],
      dtype='object')

In [19]:
df_train.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'visitStartTime',
       'device.browser', 'device.deviceCategory', 'device.isMobile',
       'device.operatingSystem', 'geoNetwork.city', 'geoNetwork.continent',
       'geoNetwork.country', 'geoNetwork.region', 'geoNetwork.subContinent',
       'totals.bounces', 'totals.hits', 'totals.newVisits', 'totals.pageviews',
       'totals.transactionRevenue'],
      dtype='object')

In [11]:
#columns to drop
to_drop = [ 
     'visitId', 'visitNumber', 
        'geoNetwork.metro',
       'geoNetwork.networkDomain', 
        'trafficSource.adContent',
        'trafficSource.campaign'        ,
       'trafficSource.isTrueDirect', 
           'trafficSource.keyword',
       'trafficSource.medium', 
           'trafficSource.referralPath'      ,
       'trafficSource.source'
          ]

In [12]:
# df_train.drop(to_drop, axis=1, inplace=True)
# df_test.drop(to_drop, axis=1, inplace=True)

KeyError: "['visitId' 'visitNumber' 'trafficSource.adContent' 'trafficSource.campaign' 'trafficSource.isTrueDirect'\n 'trafficSource.keyword' 'trafficSource.medium' 'trafficSource.referralPath' 'trafficSource.source'] not found in axis"

In [20]:
from datetime import datetime

# This function is to extract date features
def date_process(df):
    df["date"] = pd.to_datetime(df["date"], format="%Y%m%d") # setting the column as pandas datetime
    df["_weekday"] = df['date'].dt.weekday #extracting week day
    df["_day"] = df['date'].dt.day # extracting day
    df["_month"] = df['date'].dt.month # extracting day
    df["_year"] = df['date'].dt.year # extracting day
    df['_visitHour'] = (df['visitStartTime'].apply(lambda x: str(datetime.fromtimestamp(x).hour))).astype(int)
    
    return df #returning the df after the transformations

In [21]:
df_train = date_process(df_train)
df_test = date_process(df_test)

In [22]:
def NumericalColumns(df):    # fillna numeric feature
    df['totals.pageviews'].fillna(1, inplace=True) #filling NA's with 1
    df['totals.newVisits'].fillna(0, inplace=True) #filling NA's with 0
#     df['totals.transactions'].fillna(0, inplace=True) #filling NA's with 0
#     df['totals.timeOnSite'].fillna(0, inplace=True) #filling NA's with 0
    
    df['totals.bounces'].fillna(0, inplace=True)   #filling NA's with 0
#     df['trafficSource.isTrueDirect'].fillna(False, inplace=True) # filling boolean with False
#     df['trafficSource.adwordsClickInfo.isVideoAd'].fillna(True, inplace=True) # filling boolean with True
    df["totals.transactionRevenue"] = df["totals.transactionRevenue"].fillna(1.0).astype(float) #filling NA with zero
    df['totals.pageviews'] = df['totals.pageviews'].astype(int) # setting numerical column as integer
    df['totals.newVisits'] = df['totals.newVisits'].astype(int) # setting numerical column as integer
#     df['totals.bounces'] = df['totals.bounces'].astype(int)  # setting numerical column as integer
    df["totals.hits"] = df["totals.hits"].astype(float) # setting numerical to float
#     df['totals.visits'] = df['totals.visits'].astype(int) # seting as int

    return df #return the transformed dataframe

In [23]:
df_train = NumericalColumns(df_train)
df_test = NumericalColumns(df_test)


In [24]:
from sklearn import preprocessing

def Normalizing(df):
    # Use MinMaxScaler to normalize the column
#     df["totals.hits"] =  (df['totals.hits'] - min(df['totals.hits'])) / (max(df['totals.hits'])  - min(df['totals.hits']))
    # normalizing the transaction Revenue
    df['totals.transactionRevenue'] = df['totals.transactionRevenue'].apply(lambda x: np.log1p(x))
    # return the modified df
    return df 

In [25]:
df_train = Normalizing(df_train)
df_test = Normalizing(df_test)

In [26]:
df_train.head().T.head(40)

,0,1,2,3,4
channelGrouping,Organic Search,Referral,Direct,Organic Search,Organic Search
date,2017-10-16 00:00:00,2017-10-16 00:00:00,2017-10-16 00:00:00,2017-10-16 00:00:00,2017-10-16 00:00:00
fullVisitorId,3162355547410993243,8934116514970143966,7992466427990357681,9075655783635761930,6960673291025684308
visitStartTime,1.5082e+09,1.50818e+09,1.5082e+09,1.50817e+09,1.50819e+09
device.browser,Firefox,Chrome,Chrome,Chrome,Chrome
device.deviceCategory,desktop,desktop,mobile,desktop,desktop
device.isMobile,0,0,1,0,0
device.operatingSystem,Windows,Chrome OS,Android,Windows,Windows
geoNetwork.city,not available in demo dataset,Cupertino,not available in demo dataset,not available in demo dataset,not available in demo dataset
geoNetwork.continent,Europe,Americas,Americas,Asia,Americas


In [27]:
df_test.head().T.head(40)

,0,1,2,3,4
channelGrouping,Organic Search,Direct,Organic Search,Direct,Organic Search
date,2018-05-11 00:00:00,2018-05-11 00:00:00,2018-05-11 00:00:00,2018-05-11 00:00:00,2018-05-11 00:00:00
fullVisitorId,7460955084541987166,460252456180441002,3461808543879602873,975129477712150630,8381672768065729990
visitStartTime,1.5261e+09,1.52606e+09,1.52607e+09,1.52611e+09,1.52606e+09
device.browser,Chrome,Chrome,Chrome,Chrome,Internet Explorer
device.deviceCategory,mobile,desktop,desktop,mobile,tablet
device.isMobile,1,0,0,1,1
device.operatingSystem,Android,Macintosh,Chrome OS,iOS,Windows
geoNetwork.city,(not set),San Francisco,not available in demo dataset,Houston,Irvine
geoNetwork.continent,Asia,Americas,Americas,Americas,Americas


In [28]:
# ['channelGrouping', 'date', 'fullVisitorId', 'visitStartTime',
#        'device.browser', 'device.deviceCategory', 'device.isMobile',
#        'device.operatingSystem', 'geoNetwork.city', 'geoNetwork.continent',
#        'geoNetwork.country', 'geoNetwork.region', 'geoNetwork.subContinent',
#        'totals.bounces', 'totals.hits', 'totals.newVisits', 'totals.pageviews',
#        'totals.transactionRevenue', 'trafficSource.isTrueDirect',
#        'trafficSource.keyword', 'trafficSource.medium', 'trafficSource.source',
#        '_weekday', '_day', '_month', '_year', '_visitHour']
# cat_vars = ['channelGrouping', 'device.deviceCategory', 'device.isMobile',
#        'device.browser', 'device.operatingSystem','trafficSource.isTrueDirect',
#        'trafficSource.keyword', 'trafficSource.medium', 'trafficSource.source',
#             'geoNetwork.city',
#        'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.region',
#        'geoNetwork.subContinent','_weekday','_day','_month','_year','_visitHour']
# contin_vars = ['totals.hits', 'totals.newVisits','totals.bounces', 
#        'totals.pageviews']
cat_vars = ['channelGrouping', 'device.deviceCategory', 'device.isMobile',
       'device.browser', 'device.operatingSystem',
            'geoNetwork.city',
        'geoNetwork.city',
       'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.region',
       'geoNetwork.subContinent','_weekday','_day','_month','_year','_visitHour']
contin_vars = ['totals.hits', 'totals.newVisits',
       'totals.pageviews'
           ]


In [29]:
dep = 'totals.transactionRevenue'
df_train = df_train[cat_vars+contin_vars+[dep, 'date']].copy()

In [30]:
df_test[dep] = 0.0
df_test = df_test[cat_vars+contin_vars+[dep, 'date', 'fullVisitorId']].copy()

In [ ]:
for v in cat_vars: df_train[v] = df_train[v].astype('category').cat.as_ordered()

In [145]:
apply_cats(df_test, df_train)

In [146]:
for v in contin_vars:
    df_train[v] = df_train[v].fillna(0).astype('float32')
    df_test[v] = df_test[v].fillna(0).astype('float32')

In [147]:
samp_size = len(df_train)
df_samp = df_train.set_index("date")

In [148]:
df, y, nas, mapper = proc_df(df_samp, 'totals.transactionRevenue', do_scale=True)
yl = np.log(y)

In [149]:
df_test = df_test.set_index("date")

In [150]:
df_test1, _, nas, mapper = proc_df(df_test, dep, do_scale=True,skip_flds=['fullVisitorId'],mapper=mapper, na_dict=nas)

In [151]:
train_ratio = 0.75
# train_ratio = 0.9
train_size = int(samp_size * train_ratio); train_size
val_idx = list(range(train_size, len(df)))

In [152]:
df.columns

Index(['channelGrouping', 'device.deviceCategory', 'device.isMobile',
       'device.browser', 'device.operatingSystem', 'geoNetwork.city',
       'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.region',
       'geoNetwork.subContinent', '_weekday', '_day', '_month', '_year',
       '_visitHour', 'totals.hits', 'totals.newVisits', 'totals.pageviews'],
      dtype='object')

In [100]:
val_idx = np.flatnonzero(
    (df_samp.date<=datetime(2018,9,1)) & (df_samp.index>=datetime(2018,10,15)))

AttributeError: 'DataFrame' object has no attribute 'date'

In [153]:
val_idx

[1205879,
 1205880,
 1205881,
 1205882,
 1205883,
 1205884,
 1205885,
 1205886,
 1205887,
 1205888,
 1205889,
 1205890,
 1205891,
 1205892,
 1205893,
 1205894,
 1205895,
 1205896,
 1205897,
 1205898,
 1205899,
 1205900,
 1205901,
 1205902,
 1205903,
 1205904,
 1205905,
 1205906,
 1205907,
 1205908,
 1205909,
 1205910,
 1205911,
 1205912,
 1205913,
 1205914,
 1205915,
 1205916,
 1205917,
 1205918,
 1205919,
 1205920,
 1205921,
 1205922,
 1205923,
 1205924,
 1205925,
 1205926,
 1205927,
 1205928,
 1205929,
 1205930,
 1205931,
 1205932,
 1205933,
 1205934,
 1205935,
 1205936,
 1205937,
 1205938,
 1205939,
 1205940,
 1205941,
 1205942,
 1205943,
 1205944,
 1205945,
 1205946,
 1205947,
 1205948,
 1205949,
 1205950,
 1205951,
 1205952,
 1205953,
 1205954,
 1205955,
 1205956,
 1205957,
 1205958,
 1205959,
 1205960,
 1205961,
 1205962,
 1205963,
 1205964,
 1205965,
 1205966,
 1205967,
 1205968,
 1205969,
 1205970,
 1205971,
 1205972,
 1205973,
 1205974,
 1205975,
 1205976,
 1205977,
 1205978,


In [154]:
def inv_y(a): return np.exp(a)

def exp_rmspe(y_pred, targ):
    targ = inv_y(targ)
    pct_var = (targ - inv_y(y_pred))/targ
    return math.sqrt((pct_var**2).mean())

max_log_y = np.max(yl)
y_range = (0, max_log_y*1.2)

In [155]:
md = ColumnarModelData.from_data_frame(PATH, val_idx, df, yl.astype(np.float32), cat_flds=cat_vars, bs=128,
                                       test_df=df_test1)

In [156]:
cat_sz = [(c, len(df_samp[c].cat.categories)+1) for c in cat_vars]

In [157]:
cat_sz

[('channelGrouping', 9),
 ('device.deviceCategory', 4),
 ('device.isMobile', 3),
 ('device.browser', 120),
 ('device.operatingSystem', 25),
 ('geoNetwork.city', 941),
 ('geoNetwork.continent', 7),
 ('geoNetwork.country', 229),
 ('geoNetwork.region', 479),
 ('geoNetwork.subContinent', 24),
 ('_weekday', 8),
 ('_day', 32),
 ('_month', 13),
 ('_year', 4),
 ('_visitHour', 25)]

In [158]:
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]

In [159]:
emb_szs

[(9, 5),
 (4, 2),
 (3, 2),
 (120, 50),
 (25, 13),
 (941, 50),
 (7, 4),
 (229, 50),
 (479, 50),
 (24, 12),
 (8, 4),
 (32, 16),
 (13, 7),
 (4, 2),
 (25, 13)]

In [160]:
m = md.get_learner(emb_szs,len(df.columns)-len(cat_vars),0.04,1,[1000,500],[0.001,0.01])

In [161]:
m.model

MixedInputModel(
  (embs): ModuleList(
    (0): Embedding(9, 5)
    (1): Embedding(4, 2)
    (2): Embedding(3, 2)
    (3): Embedding(120, 50)
    (4): Embedding(25, 13)
    (5): Embedding(941, 50)
    (6): Embedding(7, 4)
    (7): Embedding(229, 50)
    (8): Embedding(479, 50)
    (9): Embedding(24, 12)
    (10): Embedding(8, 4)
    (11): Embedding(32, 16)
    (12): Embedding(13, 7)
    (13): Embedding(4, 2)
    (14): Embedding(25, 13)
  )
  (lins): ModuleList(
    (0): Linear(in_features=283, out_features=1000, bias=True)
    (1): Linear(in_features=1000, out_features=500, bias=True)
  )
  (bns): ModuleList(
    (0): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True)
    (1): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True)
  )
  (outp): Linear(in_features=500, out_features=1, bias=True)
  (emb_drop): Dropout(p=0.04)
  (drops): ModuleList(
    (0): Dropout(p=0.001)
    (1): Dropout(p=0.01)
  )
  (bn): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True)
)

In [ ]:
lr=3e-3
m.lr_find()

In [ ]:
m.sched.plot(100)

In [ ]:
m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars),
                   0.04, 1, [2000,1000], [0.001,0.01], y_range=y_range)
lr = 1e-3

In [123]:
# m.fit(lr, 3, metrics=[exp_rmspe])
lr=1e-4
m.fit(lr, 5, metrics=[exp_rmspe], cycle_len=1)

epoch      trn_loss   val_loss   exp_rmspe                                                                             
    0      2.172534   2.407206   1315.580934
    1      2.808699   2.357672   164849.455056                                                                         
    2      2.055517   2.345468   2057.839535                                                                           
    3      2.198851   2.338394   40705.904903                                                                          
  3%|██                                                                  | 261/8833 [00:04<02:20, 61.14it/s, loss=2.37]

KeyboardInterrupt: 

In [105]:
m.save('model-workingv3')

In [107]:
m.load('model-workingv3')

In [108]:
x,y=m.predict_with_targs()

In [109]:
exp_rmspe(x,y)

67805.84423189494

In [202]:
pred_test=m.predict(True)
pred_test = np.exp(pred_test)

In [203]:
df_test[dep]=pred_test-1

In [205]:
csv_fn=f'{PATH}/tmp/subv2.csv'

In [206]:
df_test[['fullVisitorId',dep]].to_csv(csv_fn, index=False)

In [207]:
df_try_unique = pd.read_csv(csv_fn,low_memory=False)

In [208]:
# df_try_unique = df_try_unique.drop_duplicates(['fullVisitorId'])
df_try_unique.size

803178

In [209]:
df_try_unique1 = df_try_unique.groupby('fullVisitorId').sum()

In [210]:
df_try_unique1 = df_try_unique1.reset_index()

In [211]:
# df_try_unique1 = df_try_unique1.drop_duplicates(['fullVisitorId'])
df_try_unique1.size

593060

In [212]:
csv_fn1=f'{PATH}/tmp/submission_newv2.csv'
df_try_unique1[['fullVisitorId','totals.transactionRevenue']].to_csv(csv_fn1, index=False)

In [116]:
from sklearn.ensemble import RandomForestRegressor

In [117]:
((val,trn), (y_val,y_trn)) = split_by_idx(val_idx, df.values, yl)

In [118]:
m = RandomForestRegressor(n_estimators=40, max_features=0.99, min_samples_leaf=2,
                          n_jobs=-1, oob_score=True)
m.fit(trn, y_trn);

In [119]:
val_idx

[1130511,
 1130512,
 1130513,
 1130514,
 1130515,
 1130516,
 1130517,
 1130518,
 1130519,
 1130520,
 1130521,
 1130522,
 1130523,
 1130524,
 1130525,
 1130526,
 1130527,
 1130528,
 1130529,
 1130530,
 1130531,
 1130532,
 1130533,
 1130534,
 1130535,
 1130536,
 1130537,
 1130538,
 1130539,
 1130540,
 1130541,
 1130542,
 1130543,
 1130544,
 1130545,
 1130546,
 1130547,
 1130548,
 1130549,
 1130550,
 1130551,
 1130552,
 1130553,
 1130554,
 1130555,
 1130556,
 1130557,
 1130558,
 1130559,
 1130560,
 1130561,
 1130562,
 1130563,
 1130564,
 1130565,
 1130566,
 1130567,
 1130568,
 1130569,
 1130570,
 1130571,
 1130572,
 1130573,
 1130574,
 1130575,
 1130576,
 1130577,
 1130578,
 1130579,
 1130580,
 1130581,
 1130582,
 1130583,
 1130584,
 1130585,
 1130586,
 1130587,
 1130588,
 1130589,
 1130590,
 1130591,
 1130592,
 1130593,
 1130594,
 1130595,
 1130596,
 1130597,
 1130598,
 1130599,
 1130600,
 1130601,
 1130602,
 1130603,
 1130604,
 1130605,
 1130606,
 1130607,
 1130608,
 1130609,
 1130610,


In [120]:
preds = m.predict(val)
m.score(trn, y_trn), m.score(val, y_val), m.oob_score_, exp_rmspe(preds, y_val)

(0.7944816197916722,
 0.27538311160571916,
 0.25539526769819065,
 50768.8209096924)